In [379]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json

In [534]:
def url_content(url):
    response = urlopen(url, timeout=20)
    soup = BeautifulSoup(response, "html5lib")

    
    ## extract the general information
    general_information = json.loads(soup.find('script', type="application/ld+json").text)
    
    review_date = general_information['datePublished']
    title = general_information['name']
    content = general_information['reviewBody']
    overall_rating = int(general_information['reviewRating']['ratingValue'])
    
    ##return review_date, title, content, overall_rating
    
    ## extract the traveling type and date
    stay_info = soup.find('div',{'class':"recommend-titleInline"}).text
    stay_hardcode = 'Stayed: '
    comma_hardcode = ', '    
    stay_date = stay_info[len(stay_hardcode):stay_info.find(',')]
    traveling_type = ''
    traveling_type_table = ['family','couple','solo','business','frined']
    for type_ in traveling_type_table:
        if type_ in stay_info:
            traveling_type = type_
            break
    if traveling_type == '': ## save the whole crwl info if the traveling type cannot be detacted
        try:
            print ('type cannot be detected')
            traveling_type = stay_info[len('travelled as a '):]
        except:
            traveling_type = stay_info
    ##return stay_date, traveling_type
            
    ## ranking part
    star_info = soup.findAll('div',{'class':"ui_column is-10-desktop is-12-tablet is-12-mobile"})
    star_info = star_info[0].findAll('li',{'class':"recommend-answer"})
    
    description = 'ui_bubble_rating bubble_'
    ranking_dict = dict()
    
    if len(star_info) > 0: #only do this when the reviewer provides the rating of each area
        for rate in star_info:
            rate_area = rate.find('div',{'class':"recommend-description"}).text

            pointer = str(rate).find(description) + len('ui_bubble_rating bubble_')
            try:
                rate_mark = int(str(rate)[pointer:pointer+2])
            except:
                rate_mark = str(rate)

            ranking_dict[rate_area] = rate_mark
    
    ##return ranking_dict
    
    ## return all the info
    return review_date, title, content, overall_rating, stay_date, traveling_type, ranking_dict
    

In [501]:
test_url = 'https://en.tripadvisor.com.hk/ShowUserReviews-g297701-d603335-r616402936-Viceroy_Bali-Ubud_Bali.html'
#test_url = 'https://en.tripadvisor.com.hk/ShowUserReviews-g297701-d603335-r507151028-Viceroy_Bali-Ubud_Bali.html'
#request = urllib.urllib.Request(test_url)
response = urlopen(test_url)

#content = response.read()

soup = BeautifulSoup(response.read(),'html.parser')

In [502]:
tmp = soup.findAll('div',{'class':"ui_column is-10-desktop is-12-tablet is-12-mobile"})

In [505]:
rating = tmp[0].findAll('li',{'class':"recommend-answer"})
rating

[<li class="recommend-answer"><div class="ui_bubble_rating bubble_50"></div><div class="recommend-description">Value</div></li>,
 <li class="recommend-answer"><div class="ui_bubble_rating bubble_50"></div><div class="recommend-description">Rooms</div></li>,
 <li class="recommend-answer"><div class="ui_bubble_rating bubble_50"></div><div class="recommend-description">Service</div></li>]

In [532]:
url = 'https://en.tripadvisor.com.hk/ShowUserReviews-g297701-d603335-r616402936-Viceroy_Bali-Ubud_Bali.html'

review_date, title, content, overall_rating, stay_date, traveling_type, ranking_dict = url_content(url)

stay_date: August 2018
type detected


In [535]:
print ('review date: ',review_date)
print ('review title: ',title)
print ('review content: ', content)
print ('overall rating: ', overall_rating)
print ('Stay date: ',stay_date)
print ('traveling type: ', traveling_type)
print ('Rating Table: ', ranking_dict)

review date:  14 September 2018
review title:  The most amazing place I have ever been
review content:  I would highly recommend this place.\nThe place is amazing. This was our last stop on our honeymoon and it did not disappoint. When we arrived we were given a welcome drink and then shown to the bar to sit down for 10 mins while our room was finished. During that time we saw monkeys in the trees opposite. We were then shown to our villa and again this did not disappoint, it was amazing. The monkeys we saw earlier came closer but not all the way up to our villa.\nThe food in the restaurant was great and the staff were fantastic. The spa was great.\nThere is some building work going on but it does not disturb you at all.\nWonderful place and would highly recommend. Thank you everyone at Viceroy Bali for making our trip so special.
overall rating:  5
Stay date:  August 2018
traveling type:  couple
Rating Table:  {'Value': 50, 'Service': 50, 'Rooms': 50}
